In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import data as web
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

import datetime as dt
import mplfinance as mpf
import time
import yfinance as yf

import os
from os import listdir
from os.path import isfile, join

In [2]:
#default Values
path ="/workspaces/Quantitative-Finance/Data/"
S_year = 2018
S_month = 1
S_day = 1
S_date_str = f"{S_year}-{S_month}-{S_day}"
S_date_datetime = dt.datetime(S_year,S_month,S_day)

E_year = 2020
E_month = 12
E_day = 31
E_date_str = f"{E_year}-{E_month}-{E_day}"
E_date_datetime = dt.datetime(E_year,E_month,E_day)


In [3]:
# # get stock file names in a list
# files = [x for x in listdir(path) if isfile(join(path,x))]
# tickers = [os.path.splitext(x)[0] for x in files]
# # create dataframe from our list
# stock_df = pd.DataFrame(tickers,columns=["tickers"])

# return dataframe from csv
def get_df_from_csv(ticker):
    try:
        df = pd.read_csv(path+ticker+".csv")
    except FileNotFoundError:
        print("File doesn't exist")
    else:
        return df
    

# Save dataframe to csv
def save_dataframe_to_csv(df,ticker):
    df.to_csv(path+ticker+".csv")

# return on investment over time
def get_roi(df):
    df['Date'] = pd.to_datetime(df['Date'])
    start_val = df[df['Date'] == S_date_str]['Adj Close'][0]
    end_val = df[df['Date'] == E_date_str]['Adj Close'][0]
    print("Initial Price:",start_val)
    print("Final Price:",end_val)
    roi = (end_val - start_val) / start_val
    return roi

# get coefficient of variance
def get_cov(stock_df):
    for stock in stock_df:
        mean,sd = stock_df['Adj Close'].mean(),stock_df['Adj Close'].std()
        cov = sd / mean
        return cov
    

# Merge Multiple stock in on df
def merge_df_by_column_name(col_name,*tickers):
    
    mult_df = pd.DataFrame()
    start = S_date_datetime
    end = E_date_datetime
    
    for x in tickers:
        mult_df[x] = get_df_from_csv(x)[col_name]
    return mult_df

def get_valid_dates(df, sdate, edate):  
    try:
        mask = (df['Date'] > sdate) & (df['Date'] <= edate) 
        sm_df = df.loc[mask]
        sm_df = sm_df.set_index(['Date'])
        sm_date = sm_df.index.min()
        last_date = sm_df.index.max()
        date_leading = '-'.join(('0' if len(x)<2 else '')+x for x in sm_date.split('-'))
        date_ending = '-'.join(('0' if len(x)<2 else '')+x for x in last_date.split('-'))
    except Exception:
        print("Date Corrupted")
    else:
        return date_leading, date_ending
    
def roi_between_dates(df, sdate, edate):
    try: 
        start_val = df.loc[sdate,'Adj Close'] 
        end_val = df.loc[edate,'Adj Close']
        roi = ((end_val - start_val) / start_val)
    except Exception:
        print("Data Corrupted")
    else:
        return roi
    
def merge_df_by_column_name(col_name, sdate, edate, *tickers):
    # Will hold data for all dataframes with the same column name
    mult_df = pd.DataFrame()
    
    for x in tickers:
        df = get_df_from_csv(x)
        df['Date'] = pd.to_datetime(df['Date'])
        # Use a mask to grab data between defined dates
        mask = (df['Date'] >= sdate) & (df['Date'] <= edate)
        mult_df[x] = df.loc[mask][col_name]
        
    return mult_df

In [4]:
# Get Sector Stocks
sec_df = pd.read_csv('/workspaces/Quantitative-Finance/Basic/stock_sectors.csv')
sec_df

indus_df = sec_df.loc[sec_df['Sector'] == "Industrials"]
health_df = sec_df.loc[sec_df['Sector'] == "Health Care"]
it_df = sec_df.loc[sec_df['Sector'] == "Information Technology"]
comm_df = sec_df.loc[sec_df['Sector'] == "Communication Services"]
staple_df = sec_df.loc[sec_df['Sector'] == "Consumer Staples"]
discretion_df = sec_df.loc[sec_df['Sector'] == "Consumer Discretionary"]
utility_df = sec_df.loc[sec_df['Sector'] == "Utilities"]
financial_df = sec_df.loc[sec_df['Sector'] == "Financials"]
material_df = sec_df.loc[sec_df['Sector'] == "Materials"]
restate_df = sec_df.loc[sec_df['Sector'] == "Real Estate"]
energy_df = sec_df.loc[sec_df['Sector'] == "Energy"]

In [5]:
# returns a df with roi for all stocks
def get_rois_for_stocks(stock_df):
    tickers = []
    rois = []
    for index,row in stock_df.iterrows():
        df = get_df_from_csv(row['Symbol'])
        if df is None:
            pass
        else:
            tickers.append(row['Symbol'])
            sdate, edate = get_valid_dates(df, '2018-01-01', '2020-12-31')
            df = df.set_index(['Date'])
            roi = roi_between_dates(df, sdate, edate)
            rois.append(roi)
    return pd.DataFrame({'Ticker':tickers, 'ROI':rois})

In [6]:
industrial = get_rois_for_stocks(indus_df)
industrial

File doesn't exist


File doesn't exist
File doesn't exist
File doesn't exist
File doesn't exist
File doesn't exist
File doesn't exist


,Ticker,ROI
0,MMM,-0.183592
1,AOS,-0.060680
2,ALK,-0.273133
3,ALLE,0.504415
4,AAL,-0.694661
...,...,...
62,VRSK,1.195231
63,GWW,0.831677
64,WAB,-0.100293
65,WM,0.452346


In [7]:
# Top Industries
industrial.sort_values(by=["ROI"], ascending=False).head(5)

,Ticker,ROI
24,GNRC,3.734749
10,CPRT,1.918578
57,TDG,1.518984
9,CTAS,1.325593
42,ODFL,1.202971


### Portfolio Management

In [8]:
port_list = ["GNRC",'DXCM','AMD','NFLX','COST','TGT','AES','MSCI','NEM','SBAC','HES']

In [9]:
mult_df = merge_df_by_column_name('daily_return','2018-01-01','2020-12-31',*port_list)
mult_df.corr()

,GNRC,DXCM,AMD,NFLX,COST,TGT,AES,MSCI,NEM,SBAC,HES
GNRC,1.000000,0.289526,0.395187,0.340732,0.465304,0.338411,0.433325,0.515742,0.250697,0.444762,0.426870
DXCM,0.289526,1.000000,0.328297,0.307643,0.265366,0.196806,0.218423,0.469509,0.147494,0.243462,0.243240
AMD,0.395187,0.328297,1.000000,0.459581,0.394315,0.340573,0.258974,0.485819,0.191355,0.294906,0.288337
NFLX,0.340732,0.307643,0.459581,1.000000,0.430674,0.259971,0.204159,0.449132,0.103639,0.216935,0.217857
COST,0.465304,0.265366,0.394315,0.430674,1.000000,0.500433,0.319886,0.482322,0.183141,0.455022,0.234526
TGT,0.338411,0.196806,0.340573,0.259971,0.500433,1.000000,0.305004,0.325772,0.092757,0.304365,0.245106
AES,0.433325,0.218423,0.258974,0.204159,0.319886,0.305004,1.000000,0.483399,0.176918,0.333249,0.470414
MSCI,0.515742,0.469509,0.485819,0.449132,0.482322,0.325772,0.483399,1.000000,0.258932,0.498281,0.411647
NEM,0.250697,0.147494,0.191355,0.103639,0.183141,0.092757,0.176918,0.258932,1.000000,0.275681,0.149387
SBAC,0.444762,0.243462,0.294906,0.216935,0.455022,0.304365,0.333249,0.498281,0.275681,1.000000,0.273548


In [10]:
def get_stock_price_on_date(ticker,date):
    df = get_df_from_csv(ticker)
    df = df.set_index(['Date'])
    return df.loc[date,'Adj Close']

In [11]:
print("GNRC $", get_stock_price_on_date('GNRC','2021-04-09'))

GNRC $ 330.4100036621094


In [12]:
# return portfolio weight
def get_port_weight(price, total):
    return price/total

In [13]:
gnrc_w = get_port_weight(449.12, 4905.98)
dxcm_w = get_port_weight(357.74, 4905.98)
amd_w = get_port_weight(369.2, 4905.98)
nflx_w = get_port_weight(522.86, 4905.98)
cost_w = get_port_weight(377.88, 4905.98)
tgt_w = get_port_weight(527.46, 4905.98)
aes_w = get_port_weight(410.76, 4905.98)
msci_w = get_port_weight(435.40, 4905.98)
nem_w = get_port_weight(434.49, 4905.98)
sbac_w = get_port_weight(548.84, 4905.98)
hes_w = get_port_weight(472.23, 4905.98)

wts = np.array([gnrc_w, dxcm_w, amd_w, nflx_w, cost_w, tgt_w, aes_w, msci_w, \
                nem_w, sbac_w, hes_w])
wts

port_var = np.dot(wts.T,np.dot(mult_df.cov() * 756,wts))
print("Portfolio Covariance: ",port_var)
print("GNRC Var: ", mult_df["GNRC"].var() * 756)


Portfolio Covariance:  0.18221033377623252
GNRC Var:  0.4394654360724252


In [14]:
# Diversifiable Risk
def calc_diversifiable_risk(df, tickers, weights):
    # Gets number of days
    days = len(mult_df.index)
    # Calculate covariance of portfolio
    port_covar = np.dot(weights.T, np.dot(df.cov() * days, weights)) 
    
    i = 0
    while i < len(tickers):
        wt_sq = weights[i] ** 2
        stk_var = mult_df[tickers[i]].var() * days
        wt_var = wt_sq * stk_var
        port_covar = port_covar - wt_var
        i += 1
    return port_covar

In [15]:
div_risk = calc_diversifiable_risk(mult_df, port_list, wts)
div_risk

0.13626654511704742

In [16]:
print("Systematic Risk :", (port_var - div_risk))

Systematic Risk : 0.04594378865918511
